In [47]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd

In [33]:
joined = pd.read_csv('joined')

In [103]:
first_query = pd.read_csv('first_query.txt', header=None)
first_query.rename(columns = {0:'First Name',1:'Last Name',2:'YearsOfExp', 3: 'State',4:'Zip',5:'Rating'},inplace=True)

In [35]:
merged = pd.merge(joined,first_query,on=['First Name','Last Name', 'State'], how='left')

In [36]:
merged['YearsOfExp'].fillna(0,inplace=True)
merged['Rating'].fillna(0,inplace=True)

In [37]:
merged.drop('Unnamed: 0', axis=1, inplace=True)
merged.corr()

,indem_label,BoardCredit,ClaimsBand,Procedures,JCAHO,RecordKepping,age,YearsOfExp,Rating
indem_label,1.000000,0.126012,0.107918,0.149956,0.184046,0.053250,0.104640,0.110566,0.130915
BoardCredit,0.126012,1.000000,0.505971,0.037929,0.420447,0.149109,0.096657,0.195741,0.236554
ClaimsBand,0.107918,0.505971,1.000000,0.073699,0.434657,0.216368,0.342873,0.319128,0.288525
Procedures,0.149956,0.037929,0.073699,1.000000,0.009102,-0.033182,0.040425,0.010883,0.012298
JCAHO,0.184046,0.420447,0.434657,0.009102,1.000000,0.251434,0.075669,0.172393,0.239804
RecordKepping,0.053250,0.149109,0.216368,-0.033182,0.251434,1.000000,0.003453,0.041939,0.140511
age,0.104640,0.096657,0.342873,0.040425,0.075669,0.003453,1.000000,0.453728,0.138201
YearsOfExp,0.110566,0.195741,0.319128,0.010883,0.172393,0.041939,0.453728,1.000000,0.544558
Rating,0.130915,0.236554,0.288525,0.012298,0.239804,0.140511,0.138201,0.544558,1.000000


In [75]:
# drop rows with any NaN values
# not a long-term solution, but a quick and dirty model
no_suit = merged[merged.indem_label == 0]
suit_no_indem = merged[merged.indem_label == 1]
suit_indem = merged[merged.indem_label == 2]

no_suit.dropna(how='any', inplace=True)
suit_no_indem.dropna(how='any', inplace=True)
suit_indem.dropna(how='any', inplace=True)

# print totals after NaN drop
print no_suit.shape[0], suit_indem.shape[0], suit_no_indem.shape[0]

# !
# change values to 1 for binary classification
# suit_indem['indem_label'] = 1

# random sample from the three categories
no_suit = no_suit.sample(600)
suit_no_indem = suit_no_indem.sample(164)
suit_indem = suit_indem.sample(24)

1205 24 164


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [76]:
# combine random sample dataframes into concat_df
concat_df = pd.concat([no_suit, suit_indem, suit_no_indem], axis=0)

In [ ]:
# build label and feature dataframes
y = concat_df.pop('indem_label')

In [93]:
to_model = concat_df[['YearsOfExp','Rating','BoardCredit','Procedures','JCAHO','age']]
state = concat_df.pop('State')
dummies = pd.get_dummies(state)
to_model = pd.concat([to_model,dummies],axis=1)

In [94]:
# train / test split with random state
X_train, X_test, y_train, y_test = train_test_split(to_model, y, test_size=0.20)

In [95]:
model = RandomForestClassifier(n_estimators=64,oob_score=True)

In [99]:
# fit, validate with OOB score
model.fit(X_train,y_train)
model.oob_score_

0.76031746031746028

In [100]:
preds = model.predict(X_test)
model.score(X_test, y_test)

0.69620253164556967

In [101]:
confusion_matrix(y_test, preds)

array([[106,   8,   0],
       [ 34,   4,   1],
       [  3,   2,   0]])

AttributeError: 'numpy.ndarray' object has no attribute 'proba'